In [1]:
import os
import cv2
import json
from tqdm import tqdm
import mediapipe as mp

2025-01-30 07:52:12.982796: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-01-30 07:52:13.051230: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1738223533.082522   30905 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1738223533.093458   30905 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-01-30 07:52:13.153765: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instr

In [2]:
# Mediapipe Pose setup for body and hands
mp_pose = mp.solutions.pose
pose = mp_pose.Pose(static_image_mode=True, min_detection_confidence=0.5, min_tracking_confidence=0.5)

mp_hands = mp.solutions.hands
hands = mp_hands.Hands(static_image_mode=True, max_num_hands=2, min_detection_confidence=0.5)


libEGL warning: MESA-LOADER: failed to open zink: /usr/lib/dri/zink_dri.so: cannot open shared object file: No such file or directory (search paths /usr/lib/x86_64-linux-gnu/dri:\$${ORIGIN}/dri:/usr/lib/dri, suffix _dri)

libEGL warning: MESA-LOADER: failed to open swrast: /usr/lib/dri/swrast_dri.so: cannot open shared object file: No such file or directory (search paths /usr/lib/x86_64-linux-gnu/dri:\$${ORIGIN}/dri:/usr/lib/dri, suffix _dri)

libEGL warning: MESA-LOADER: failed to open zink: /usr/lib/dri/zink_dri.so: cannot open shared object file: No such file or directory (search paths /usr/lib/x86_64-linux-gnu/dri:\$${ORIGIN}/dri:/usr/lib/dri, suffix _dri)

libEGL warning: MESA-LOADER: failed to open swrast: /usr/lib/dri/swrast_dri.so: cannot open shared object file: No such file or directory (search paths /usr/lib/x86_64-linux-gnu/dri:\$${ORIGIN}/dri:/usr/lib/dri, suffix _dri)

libEGL warning: MESA-LOADER: failed to open zink: /usr/lib/dri/zink_dri.so: cannot open shared object fi

In [3]:
# Initialize output paths
IMAGE_PATH = '../dataset/augmented_images'  # Path to your dataset
OUTPUT_PATH = '../dataset/keypoints'  # Path to save JSON files

In [4]:
def create_dir(directory_path):
    """Creates a directory if it does not already exist."""
    os.makedirs(directory_path, exist_ok=True)

In [5]:
def validate_json(output_path, label):
    """
    Validates the JSON file for the given label and prints the total number of images.
    
    Args:
        output_path (str): Path to the folder containing JSON files.
        label (str): Label of the folder (e.g., 'A', '1').
    """
    json_path = os.path.join(output_path, f"{label}.json")
    
    # Check if the file exists
    if os.path.exists(json_path):
        with open(json_path, 'r') as json_file:
            data = json.load(json_file)
            total_images = len(data)
            print(f"Total images written for label {label}: {total_images}")
    else:
        print(f"JSON file not found for label {label}: {json_path}")

In [6]:
def extract_body_keypoints(image_path):
    """
    Extracts keypoints for the body from the image.
    
    Args:
        image_path (str): Path to the image.
    
    Returns:
        list: A list of keypoints (33 keypoints for full body pose).
    """
    image = cv2.imread(image_path)
    image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    results = pose.process(image_rgb)

    keypoints_data = []

    if results.pose_landmarks:
        for landmark in results.pose_landmarks.landmark:
            # Append each x, y, z for each keypoint (normalized coordinates)
            keypoints_data.append({"x": landmark.x, "y": landmark.y, "z": landmark.z})

    # Ensure we have 33 keypoints (if any keypoint is missing, it is padded with zeros)
    if len(keypoints_data) < 33:
        keypoints_data.extend([{"x": 0.0, "y": 0.0, "z": 0.0}] * (33 - len(keypoints_data)))  # Pad with zeros if less than 33 keypoints

    return keypoints_data

W0000 00:00:1738223537.591102   31007 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.


In [7]:
def extract_hand_keypoints(image_path):
    """
    Extracts keypoints for the hands from the image.
    
    Args:
        image_path (str): Path to the image.
    
    Returns:
        list: A list of keypoints (21 keypoints per hand, up to two hands).
    """
    image = cv2.imread(image_path)
    image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    results = hands.process(image_rgb)

    keypoints_data = []

    if results.multi_hand_landmarks:
        for hand_landmarks in results.multi_hand_landmarks:
            for lm in hand_landmarks.landmark:
                keypoints_data.append({"x": lm.x, "y": lm.y, "z": lm.z})  # Append each x, y, z for each keypoint

    # Ensure we have 42 keypoints (21 per hand, up to two hands detected)
    if len(keypoints_data) < 42:
        keypoints_data.extend([{"x": 0.0, "y": 0.0, "z": 0.0}] * (42 - len(keypoints_data)))  # Pad with zeros if less than 42 keypoints

    return keypoints_data

In [8]:
def process_images_with_validation(base_path, output_path):
    """
    Processes images to extract body and hand keypoints, then saves them as JSON files.
    
    Args:
        base_path (str): Path to the base folder containing images.
        output_path (str): Path to the folder to save JSON files.
    """
    create_dir(output_path)

    # Label map for directories (you can customize this if needed)
    labels = list("123456789ABCDEFGHIJKLMNOPQRSTUVWXYZ")

    # Initialize a dictionary to hold data for each label
    label_data = {label: [] for label in labels}

    for label in tqdm(labels, desc="Processing labels"):
        input_folder = os.path.join(base_path, label)

        if not os.path.exists(input_folder):
            print(f"Folder not found: {input_folder}")
            continue

        for filename in os.listdir(input_folder):
            if filename.lower().endswith(('.jpg', '.jpeg', '.png')):
                image_path = os.path.join(input_folder, filename)

                # Extract keypoints for both body and hands
                body_keypoints = extract_body_keypoints(image_path)
                hand_keypoints = extract_hand_keypoints(image_path)

                # Create the formatted dictionary for the image
                image_data = {
                    "image_name": filename,
                    "keypoints": {
                        "body": body_keypoints,  # Store body keypoints
                        "hands": hand_keypoints   # Store hand keypoints (empty if no hands detected)
                    }
                }

                # Append the data to the label's list
                label_data[label].append(image_data)

        # Save the accumulated data for the label to a JSON file
        json_path = os.path.join(output_path, f"{label}.json")
        with open(json_path, 'w') as json_file:
            json.dump(label_data[label], json_file, indent=4)
        
        # Validate and print total images written
        validate_json(output_path, label)

W0000 00:00:1738223537.619681   31007 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.


In [9]:
if __name__ == "__main__":
    process_images_with_validation(IMAGE_PATH, OUTPUT_PATH)
    print("Keypoint extraction completed.")

Processing labels:   0%|          | 0/35 [00:00<?, ?it/s]W0000 00:00:1738223537.639028   30991 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1738223537.678774   30998 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1738223537.725648   31001 landmark_projection_calculator.cc:186] Using NORM_RECT without IMAGE_DIMENSIONS is only supported for the square ROI. Provide IMAGE_DIMENSIONS or use PROJECTION_MATRIX.
Processing labels:   3%|▎         | 1/35 [05:00<2:50:10, 300.31s/it]

Total images written for label 1: 5185


Processing labels:   6%|▌         | 2/35 [10:00<2:45:04, 300.13s/it]

Total images written for label 2: 5000


Processing labels:   9%|▊         | 3/35 [14:51<2:37:56, 296.14s/it]

...

Processing labels:  97%|█████████▋| 34/35 [2:54:11<05:11, 311.27s/it]

Total images written for label Y: 5000


Processing labels: 100%|██████████| 35/35 [2:59:20<00:00, 307.43s/it]

Total images written for label Z: 5000


Keypoint extraction completed.
